In [13]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.integrate import cumtrapz

# Constants
n0 = 0.16  # fm^{-3}
n1 = 1.1 * n0  # fm^{-3}
mu1 = 939.0  # MeV
p1 = 2.08  # MeV/fm^3
Gamma =  3.113855 # Example polytropic index (sampled from [1.77, 3.23])
cs1_sq = Gamma - 1  # c_{s,1}^2 at matching point

# Parameters for pQCD
c1 = 0.9008
d1 = 0.5034
d2 = 1.452
nu1 = 0.3553
nu2 = 0.9101

# Generate random parameters for sound speed
np.random.seed(42)  # For reproducibility
N_segments = 7
cs_max_sq = np.random.uniform(0, 1)

# Sample chemical potential breakpoints (mu_i) and sound speed squared (cs_sq_i)
mu_nodes = np.concatenate([
    [mu1],
    np.sort(np.random.uniform(mu1, 2600, N_segments - 1)),
    [2600.0]
])
cs_sq_nodes = np.concatenate([
    [cs1_sq],
    np.random.uniform(0, cs_max_sq, N_segments)
])

# Create a fine grid in mu for integration
mu_grid = np.linspace(mu1, 2600, 1000)

# Piecewise linear c_s^2(mu)
cs_sq_func = interp1d(
    mu_nodes, cs_sq_nodes, kind='linear', 
    bounds_error=False, fill_value="extrapolate"
)
cs_sq_grid = cs_sq_func(mu_grid)

# Avoid division by zero and ensure cs_sq > 0
cs_sq_grid = np.clip(cs_sq_grid, 1e-5, 1.0)

# Compute n(mu): d(ln n)/dmu = 1/(mu * c_s^2)
integrand_n = 1.0 / (mu_grid * cs_sq_grid)
log_n = np.log(n1) + cumtrapz(integrand_n, mu_grid, initial=0)
n_grid = np.exp(log_n)

# Compute p(mu): dp/dmu = n
p_grid = p1 + cumtrapz(n_grid, mu_grid, initial=0)

# Compute energy density e(mu) = mu * n - p
e_grid = mu_grid * n_grid - p_grid

# pQCD constraint at mu = 2600 MeV
X = np.random.uniform(1, 4)
mu_pQCD = 2600.0  # MeV
mu_pQCD_GeV = mu_pQCD / 1000.0  # Convert to GeV
p_pQCD = (mu_pQCD**4 / (108 * np.pi**2)) * (
    c1 - d1 * X**(-nu1) / (mu_pQCD_GeV - d2 * X**(-nu2))
)

# Check consistency (e.g., within 20%)
p_actual = p_grid[-1]
if np.abs(p_actual - p_pQCD) / p_pQCD > 0.1:
    print("EoS rejected by pQCD constraint")
else:
    print("EoS accepted")

# Output for n in (1.1n0, 40n0] -> n <= 40*n0 = 6.4 fm^{-3}
mask = (n_grid > n1) & (n_grid <= 40 * n0)
output_mu = mu_grid[mask]
output_n = n_grid[mask]
output_p = p_grid[mask]
output_e = e_grid[mask]
output_cs_sq = cs_sq_grid[mask]

EoS rejected by pQCD constraint


C:\Users\User\AppData\Local\Temp\ipykernel_8464\3502399212.py:51: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  log_n = np.log(n1) + cumtrapz(integrand_n, mu_grid, initial=0)
C:\Users\User\AppData\Local\Temp\ipykernel_8464\3502399212.py:55: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  p_grid = p1 + cumtrapz(n_grid, mu_grid, initial=0)
